# Amplitude Amplification Operators

In the *01_Dataloading_Module_Use.ipynb* and  *02_QuantumMultiplexors_Module_Use.ipynb* two different methods for implementation of the operators for loading probabilitiy $p(x)$($\mathcal{P}$) and function $f(x)$ ($\mathcal{R}$) were presented. 

With this 2 operators we want to implement a **Groover** operator mandatory for amplitude amplification strategies needed for computing the expected value of the function $f(x)$ when $x$ follows a probability distribution $f(x)$: $E_{x\sim p}(f)$.

In the **amplitude_amplification.py** script all functions for creating this **Grover** operator are implemented. 
In this notebook we review the mandatory steps for creating this operator and show how to use the code inside this script.

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import sys
sys.path.append("../library")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from copy import deepcopy

In [ ]:
%load_ext qat.core.magic
%matplotlib inline

In [ ]:
global_qlmaas = True
try:
    from qlmaas.qpus import LinAlg
except (ImportError, OSError) as e:
    global_qlmaas = False
from qat.qpus import PyLinalg

In [ ]:
def get_qpu(qlmass=False):
    """
    Function for selecting solver. User can chose between:
    * LinAlg: for submitting jobs to a QLM server
    * PyLinalg: for simulating jobs using myqlm lineal algebra.

    Parameters
    ----------

    qlmass : bool
        If True  try to use QLM as a Service connection to CESGA QLM
        If False PyLinalg simulator will be used

    Returns
    ----------
    
    lineal_qpu : solver for quantum jobs
    """
    if qlmass:
        if global_qlmaas:
            print('Using: LinAlg')
            linalg_qpu = LinAlg()
        else:
            raise ImportError("""Problem Using QLMaaS.Please create config file or use mylm solver""")
            
    else:
        print('Using PyLinalg')
        linalg_qpu = PyLinalg()
    return linalg_qpu

In [ ]:
#QLMaaS == False -> uses PyLinalg
#QLMaaS == True -> try to use LinAlg
QLMaaS = True
linalg_qpu = get_qpu(QLMaaS)

## 1. Data Discretization

The operators $\mathcal{P}$ and $\mathcal{R}$ need discretized arrays of the probability and the function to be loaded into the quantum state.

In [ ]:
from AuxiliarFunctions import  get_histogram
from data_extracting import get_results
def p(x):
    return x*x
def f(x):
    return np.sin(x)

In [ ]:
#number of Qbits for the circuit
n_qbits = 5
#The number of bins 
m_bins = 2**n_qbits
LowerLimit = 0.0
UpperLimit = 1.0 

X, p_X = get_histogram(p, LowerLimit, UpperLimit, m_bins)
f_X = f(X)

## 2. Complete Data Loading

First of all we need to loading the complete data into the quantum state:

$$|\Psi\rangle=\hat{R_{n+1}}\left(\hat{P_{n}}\otimes I\right)|0\rangle_{n}\otimes|0\rangle_{1}$$

where $|\Psi\rangle$ is the notation for a n+1 qbit quantum state

From module **QuantumMultiplexors_Module** we can configurate necesary gates:
* $\mathcal{P}$ with load_p_gate
* $\mathcal{R}$ with load_r_gate
* $\mathcal{PR}$ with load_pr_gate

In [ ]:
from QuantumMultiplexors_Module import load_p_gate, load_f_gate, load_pf_gate

In [ ]:
p_gate = load_p_gate(p_X)
f_gate = load_f_gate(f_X)
pf_gate = load_pf_gate(p_gate, f_gate)

In [ ]:
%qatdisplay p_gate --depth 1
%qatdisplay f_gate --depth 1
%qatdisplay pf_gate --depth 1

# 3. Amplitude Amplification

The quantum state resulting from the loading proccess $|\Psi\rangle$ can be expressed as a linear combination of two orthogonal states $|\Psi_{1}\rangle$ y $|\Psi_{0}\rangle$:

$$|\Psi\rangle=\sqrt{a}|\Psi_{1}\rangle+\sqrt{1-a}|\Psi_{0}\rangle$$

Where $|\Psi_{0}\rangle$ and $|\Psi_{1}\rangle$ are the following orthonormal states:

$$|\Psi_{0}\rangle = \sum_{x=0}^{2^{n}-1}|x\rangle^{n}\otimes\frac{1}{\sqrt{1-a}}\sqrt{p(x)(1-f(x))}|0\rangle$$
$$|\Psi_{1}\rangle = \sum_{x=0}^{2^{n}-1}|x\rangle^{n}\otimes\frac{1}{\sqrt{a}}\sqrt{p(x)f(x)}|1\rangle$$


Where $|x\rangle^{n}$ is a notation for a quantum state of n qbits.


The idea behind Quantum Phase Amplification is increase the probability of the $|\Psi_{1}\rangle$  using a strategy based on the Groover Algorithm. 

An Operator $\hat{Q}$ should be applied an optimal number of times $k$ in order to maximize probability of measure $|\Psi_{1}\rangle$. This operator is: 

$$\hat{Q}=\hat{U}_{|\Psi\rangle} \hat{U}_{|\Psi_{0}\rangle}$$

Where $\hat{U}_{|\Psi\rangle}$ y $\hat{U}_{|\Psi_{0}\rangle}$ are:

$$\hat{U}_{|\Psi_{0}\rangle } = \hat{I} - 2|\Psi_{0}\rangle \langle \Psi_{0}|$$
$$\hat{U}_{|\Psi\rangle } = \hat{I} - 2|\Psi\rangle\langle \Psi|$$

Module **amplitude_amplification** have all the functions in order to create properly gates for this operators

In [ ]:
#Getting the complete initial state
Phi_State, circuit, q_prog, job = get_results(pf_gate, linalg_qpu=linalg_qpu, shots=0)
#For getting the state of the additional qbit where the desired integral are stored
Initial_State, circuit, q_p, job = get_results(pf_gate, linalg_qpu=linalg_qpu, shots=0, qubits=[pf_gate.arity-1])

In [ ]:
#Complete state
Phi_State.head()

In [ ]:
#State of the auxiliar qbit
Initial_State

### 3.1 Operador $\hat{U}_{|\Psi_{0}\rangle}$

First mandatory operator is:

$$\hat{U}_{|\Psi_{0}\rangle } = \hat{I} - 2|\Psi_{0}\rangle \langle \Psi_{0}|$$

Equivalent circuit for the operator is:

$$\hat{U}_{|\Psi_{0}\rangle }=(\hat{I_{n}}\otimes X)(\hat{I_{n}}\otimes Z)(\hat{I_{n}}\otimes X)$$

Apply this operator on state $|\Psi\rangle$:

$$\hat{U}_{|\Psi_{0}\rangle} |\Psi\rangle = \sqrt{a}|\Psi_{1}\rangle-\sqrt{1-a}|\Psi_{0}\rangle$$

So operator $\hat{U}_{|\Psi_{0}\rangle }$ do a reflection operation around the axis defined by the state $|\Psi_{1}\rangle$


Abstract Gate **UPhi0_Gate** from **amplitude_amplification** allow us do the operation:

In [ ]:
from amplitude_amplification import uphi0_gate

In [ ]:
UPhi0_Gate = uphi0_gate(pf_gate.arity)
%qatdisplay UPhi0_Gate --depth 0

In [ ]:
#Apply the UPhi0_Gate to the data loading circuit
qProg_Uphi0 = deepcopy(q_prog)
registers = qProg_Uphi0.registers
qProg_Uphi0.apply(UPhi0_Gate, registers)
UPhi0_State, circuit, _, _ = get_results(qProg_Uphi0, linalg_qpu=linalg_qpu, shots=0)

In [ ]:
%qatdisplay circuit --depth 1

In [ ]:
UPhi0_State.sort_values('Int_lsb')

For testing the operator we need to check that: 
* Sates with final qbit $|0\rangle$ ($|\Psi_{0}\rangle$) change the sign with respect to the initial state $|\Psi\rangle$ 
* States with final qbit in $|1\rangle$ ($|\Psi_{1}\rangle$) are the same that in the initial state $|\Psi\rangle$

In [ ]:
#Testing Final qbit |0> should be of different sign
LastQbit0 = np.isclose(
    np.array([p for s,p in zip(Phi_State['States'], Phi_State['Amplitude']) if s.bitstring[-1] == '0']),
    -np.array([p for s,p in zip(UPhi0_State['States'], UPhi0_State['Amplitude']) if s.bitstring[-1] == '0'])
).all()
#Testing Final qbit |1> should be of same sign
LastQbit1 = np.isclose(
    np.array([p for s,p in zip(Phi_State['States'], Phi_State['Amplitude']) if s.bitstring[-1] == '1']),
    np.array([p for s,p in zip(UPhi0_State['States'], UPhi0_State['Amplitude']) if s.bitstring[-1] == '1'])
).all()
print('Test OK: {}'.format((LastQbit0 and LastQbit1)))

### 3.2 Operador $\hat{U}_{|\Psi\rangle}$

Operator $\hat{U}_{|\Psi\rangle}$ is based in Groover's difusor:

$$\hat{U}_{|\Psi\rangle } = \hat{I} - 2|\Psi\rangle\langle \Psi|$$

Additionally we know that:

$$|\Psi\rangle=\hat{R_{n+1}}\left(\hat{P_{n}}\otimes I\right)|0\rangle^{\otimes n}\otimes|0\rangle$$

Then the operator can be decomposed in the following way:

$$\hat{U}_{|\Psi\rangle } =\hat{R_{n+1}}\left(\hat{P_{n}}\otimes I\right)\hat{D}_{0} \left(\hat{P_{n}}\otimes I\right)^{\dagger} \hat{R_{n+1}}^{\dagger}$$

Where $\hat{D}_{0}$ is a reflection around the perpendicular state to $|0^{n+1}\rangle$

$$\hat{D}_{0} = \hat{I}-2|0^{n+1}\rangle\langle0^{n+1}|$$

In this case $|0^{n+1}\rangle$ is the zero state for n+1 qbits

#### 3.2.1 Implementación  $\hat{D}_{0}$

The equivalent circuit for $\hat{D}_{0}$ is:

$$\hat{D}_{0} = \hat{I}-2|0^{n+1}\rangle\langle0^{n+1}|= \hat{X}^{\otimes n+1} c^{n}Z \hat{X}^{\otimes n+1}$$

The *AbstractGate* **D0_Gate** from **amplitude_amplification** allow us create the operator:

In [ ]:
from amplitude_amplification import d0_gate

In [ ]:
D0 = d0_gate(pf_gate.arity)
%qatdisplay D0 --depth 1

In [ ]:
#Apply the d0_gate to the data loading circuit
qProg_D0 = deepcopy(q_prog)
registers = qProg_D0.registers
qProg_D0.apply(D0, registers)
UD0_State, circuit, _, _ = get_results(qProg_D0, linalg_qpu=linalg_qpu, shots=0)

In [ ]:
%qatdisplay circuit --depth 1

In [ ]:
UD0_State

Operator $\hat{D}_{0}$ over state  $|\Psi\rangle$:

$$\hat{D}_{0}|\Psi\rangle = \hat{I}|\Psi\rangle-2|0^{n+1}\rangle\langle0^{n+1}|\Psi\rangle^{n+1}$$


It can be demostrated that the only difference between $\hat{D}_{0}|\Psi\rangle$ and $|\Psi\rangle$ is that the component $|0^{n+1}\rangle$ the sign changes!!

In [ ]:
#Testing
C01 = np.isclose(Phi_State['Amplitude'].loc[0], -UD0_State['Amplitude'].loc[0])
C02 = np.isclose(Phi_State['Amplitude'].loc[1:], UD0_State['Amplitude'].loc[1:]).all()
print(C01 and C02)

#### 3.2.2 Implementation  $\hat{U}_{|\Psi}\rangle$

Finally the function **Load_UPhi_Gate** from **amplitude_amplification** creates a customized AbstractGate that implements $\hat{U}_{|\Psi\rangle }$. This functions needs the initial $\mathcal{P}$ y $\mathcal{R}$ gates used for loading the data.

This operator is reflection around the ortoghonal state to $|\Psi\rangle$

In [ ]:
from amplitude_amplification import load_uphi_gate

In [ ]:
UPhi = load_uphi_gate(pf_gate)
%qatdisplay UPhi --depth 1

In [ ]:
#Apply the UPhi to the data loading circuit
qProg_Diff = deepcopy(q_prog)
registers = qProg_Diff.registers
qProg_Diff.apply(UPhi, registers)
DiffPhi_State, circuit, _, _ = get_results(qProg_Diff, linalg_qpu=linalg_qpu, shots=0)

In this case 
$$\hat{U}_{|\Psi\rangle } |\Psi\rangle = \hat{I}|\Psi\rangle - 2|\Psi\rangle \langle \Psi|\Psi\rangle$$

$$\hat{U}_{|\Psi\rangle } |\Psi\rangle = |\Psi\rangle - 2|\Psi\rangle = -|\Psi\rangle$$



In [ ]:
#Testing
np.isclose(DiffPhi_State['Amplitude'], - Phi_State['Amplitude']).all()

## 4. Operador $\hat{Q}$

Finally we can implement the desired Operator $\hat{Q}$ 

$$\hat{Q}=\hat{U}_{|\Psi\rangle} \hat{U}_{|\Psi_{0}\rangle}$$

This operator is created using the function **Load_Q_Gate** from **amplitude_amplification**



In [ ]:
from amplitude_amplification import load_q_gate

In [ ]:
q_gate = load_q_gate(pf_gate)
%qatdisplay q_gate --depth 1

In [ ]:
#Apply the Q Grover-like operator to the data loading circuit
qProg_Q = deepcopy(q_prog)
registers = qProg_Q.registers
qProg_Q.apply(q_gate, registers)
QPhi_State, circuit, _, _ = get_results(qProg_Q, linalg_qpu=linalg_qpu, shots=0, qubits = [q_gate.arity-1])

In [ ]:
#Complete circuit
%qatdisplay circuit --depth 1

In [ ]:
QPhi_State

### Testing $\hat{Q}$ operator

To test if operator was implemented properly we know that the quantum state resulting from the complete loading proccess $|\Psi\rangle_{n+1}$ can be expressed as a linear combination of twor othogonal states $|\Psi_{1}\rangle$ and $|\Psi_{0}\rangle$:

$$|\Psi\rangle=\sqrt{a}|\Psi_{1}\rangle+\sqrt{1-a}|\Psi_{0}\rangle$$

where:

$$a = \sum_{x=0}^{2^{n}-1} p(x)f(x)$$

Making the following identities:

* $\sqrt{a} = \sin{\theta}$
* $\sqrt{1-a} = \cos{\theta}$
* $ a = \sum_{n=0}^{2^{n}-1} p(x)f(x) = \sin^{2}{\theta}$

Then:

$$|\Psi\rangle=\sin{\theta}|\Psi_{1}\rangle+\cos{\theta}|\Psi_{0}\rangle$$

So using the initial state we are going to calculate $\theta$.

In [ ]:
#First get the Amplitudes for Phi state
a0 = np.sqrt(Initial_State.iloc[0]['Probability'])
a1 = np.sqrt(Initial_State.iloc[1]['Probability'])

In [ ]:
#Calculating Theta using the quantum state from loading data: LResults
def GetAngle(Array):
    Modulo = np.linalg.norm(Array)
    cosTheta = Array[0]/Modulo
    Theta0 = np.arccos(cosTheta)
    sinTheta = Array[1]/Modulo
    Theta1 = np.arcsin(sinTheta)
    #print(Modulo, cosTheta, sinTheta, Theta0, Theta1)
    return Theta0

#Quantum state after loading data: |Psi>
Psi_ = np.array([a0, a1])
#Angle between |Psi> and axis |Psi_0>
theta = GetAngle(Psi_)
print('theta: {}'.format(theta))
print('Psi_: {}'.format(Psi_))

Now we have $\theta$ and the $|\Psi\rangle$

In [ ]:
#In order to test that this theta is ok we can compute it from p(x) and f(x)
a = sum(p_X*f_X)
sqrt_a = np.sqrt(a)
theta2 = np.arccos(sqrt_a)
print('theta2: {}'.format(theta2))

print('Is theta equals to theta2: {}'.format(np.isclose(theta, theta2)))


The operator $\hat{Q}$ can be viewed as a Y-Rotation of $\theta$ on $|\Psi\rangle$: 

$$\hat{Q}|\Psi\rangle= \hat{R}_{y}(2*\theta)|\Psi\rangle=\hat{R}_{y}(2*\theta)(\sin{\theta}|\Psi_{1}\rangle+\cos{\theta}|\Psi_{0}\rangle) = \sin{3\theta}\Psi_{1}\rangle+\cos{3\theta}|\Psi_{0}\rangle$$

So starting from $|\Psi\rangle$ we can obtain the angle $\theta$

In [ ]:
#Create a Rotation of 2*theta
c, s = np.cos(2*theta), np.sin(2.*theta)
#Rotation matrix
R = np.array(((c, -s), (s, c)))
#Apply Ry(2*theta) to quantum state |Psi>
RotationResults = np.dot(R, Psi_)

In [ ]:
RotationResults

In [ ]:
print('Square Of Rotated Vector: {}'.format(RotationResults**2))
print('Probabilities for QPhi_State: {} '.format(list(QPhi_State['Probability'])))
print('Square of the RotateState equal to Probabilities of Q|Phi> state : {}'.format(
    np.isclose(RotationResults**2, QPhi_State['Probability']).all())
     )

In [ ]:
print('Test OK: {}'.format(np.isclose(RotationResults**2, QPhi_State['Probability']).all()))

## 5. Operator $\hat{Q}^n$

In the script **amplitude_amplification.py** a **load_qn_gate** function was programed. This function receives an input gate and apply it a desired number of times

In [ ]:
from amplitude_amplification import load_qn_gate

In [ ]:
q_n_gate = load_qn_gate(q_gate, 4)
%qatdisplay q_n_gate --depth 1